In [1]:
import csv
import numpy as np
import pandas as pd
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, log_loss

# --- 1. 數據載入與英文文本預處理 ---
# 儲存不同文本輸入模式的處理結果
documents_title_only = []      # 僅包含英文標題的文本列表
documents_title_plus_content = [] # 包含英文標題+內容的文本列表
labels = []                    # 儲存數值標籤（對兩種模式共用）

print("開始載入數據並準備英文文本（標題與標題+內容）...")
file_path = '/kaggle/input/taiwan-political-news-dataset/news_training_with_translations.csv'

try:
    with open(file_path, newline='', encoding='utf-8') as f:
        rows = csv.reader(f)
        header = next(rows) # 讀取標頭
        
        # 從標頭確定欄位索引
        try:
            title_en_idx = header.index('title_en')
            content_en_idx = header.index('content_en')
            label_encoded_idx = header.index('label_encoded')
        except ValueError as e:
            print(f"錯誤：CSV 標頭中缺少預期的欄位: {e}")
            print("請確保 'title_en', 'content_en' 和 'label_encoded' 欄位存在。")
            exit() # 如果關鍵欄位遺失，程式終止

        for i, row in enumerate(rows):
            # 取得英文標題和內容
            title_en = row[title_en_idx]
            content_en = row[content_en_idx]
            
            # 取得標籤
            try:
                label = int(row[label_encoded_idx])
            except (ValueError, IndexError):
                print(f"警告：跳過第 {i+2} 行（數據行索引 {i}），因為標籤無效或缺失: {row}")
                continue # 跳過標籤有問題的行

            # 準備「僅標題」的文本
            processed_title = ' '.join([word for word in title_en.split() if len(word) > 1])
            
            # 準備「標題 + 內容」的文本
            full_text_en = title_en + ' ' + content_en
            processed_title_plus_content = ' '.join([word for word in full_text_en.split() if len(word) > 1])

            # 只有當兩種文本模式都能產生有效內容時才加入數據集
            if processed_title and processed_title_plus_content: 
                documents_title_only.append(processed_title)
                documents_title_plus_content.append(processed_title_plus_content)
                labels.append(label)
            else:
                print(f"警告：第 {i+2} 行的標題或內容處理後為空，已跳過。")

except FileNotFoundError:
    print(f"錯誤：找不到檔案 '{file_path}'。請檢查檔案路徑是否正確。")
    exit()
except Exception as e:
    print(f"讀取檔案時發生錯誤：{e}")
    exit()

print(f"數據載入與英文文本準備完成。共 {len(labels)} 篇新聞。")
print("--------------------------------------------------")

# 將標籤轉換為 NumPy 陣列
y = np.array(labels)

# 定義要測試的模型
models = {
    'LogisticRegression': LogisticRegression(random_state=42, solver='liblinear', multi_class='ovr', max_iter=2000),
    'SVC': SVC(random_state=42, probability=True), # probability=True 用於計算 log_loss
    'XGBoost': XGBClassifier(random_state=42, eval_metric='mlogloss', objective='multi:softprob') # 移除 use_label_encoder
}

# 定義模型名稱到輸出實驗名稱的映射
model_exp_name_map = {
    'LogisticRegression': 'Logistic Regression',
    'SVC': 'SVM',
    'XGBoost': 'XGBoost'
}

# 定義用於排序的各個部分的**列表**，以確保明確的順序和正確的長度
model_display_order_list = ['LogisticRegression', 'SVC', 'XGBoost'] # 內部模型鍵的排序
mode_display_order_list = ['title', 'title+content'] # 模式鍵的排序

# 建立從內部鍵到排序值的映射
model_sort_values = {name: i for i, name in enumerate(model_display_order_list)}
mode_sort_values = {name: i for i, name in enumerate(mode_display_order_list)}


# 定義兩種文本輸入模式及其對應的數據
data_modes = {
    'title': documents_title_only,
    'title+content': documents_title_plus_content
}

# 準備所有結果的列表
all_results = []

# 設定 5 摺分層交叉驗證
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) # random_state 確保結果可重現

print(f"開始對每個文本模式和每個模型執行 {n_splits} 摺分層交叉驗證...")

# Store results temporarily for proper per-fold grouping and sorting
temp_results_per_fold = {f: [] for f in range(n_splits)}


for model_key, model_instance_template in models.items(): # 使用 model_key 來獲取原始模型名稱
    # 根據映射取得要顯示在 exp 欄位中的名稱
    exp_model_display_name = model_exp_name_map.get(model_key, model_key) # 如果沒有映射則使用原始名稱

    for mode_name, documents in data_modes.items():
        print(f"\n--- 正在處理模型: {exp_model_display_name}, 文本模式: {mode_name} ---")

        # --- 2. TF-IDF 特徵提取（針對當前文本模式）---
        print(f"  開始建立 TF-IDF 向量（針對 {mode_name} 文本）...")
        tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, stop_words='english')
        X_mode = tfidf_vectorizer.fit_transform(documents) # X_mode 為當前模式的 TF-IDF 特徵矩陣
        print(f"  TF-IDF 向量建立完成。詞彙量: {len(tfidf_vectorizer.vocabulary_)}")
        print(f"  特徵矩陣形狀（文件數, 特徵數）: {X_mode.shape}")
        print("  --------------------------------------------------")

        # 複製模型實例，以確保每個實驗回合都是從一個乾淨的模型開始
        model_instance = model_instance_template.__class__(**model_instance_template.get_params())

        for fold, (train_index, test_index) in enumerate(skf.split(X_mode, y)):
            start_time_fold = time.time()

            X_train, X_test = X_mode[train_index], X_mode[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # 訓練模型
            model_instance.fit(X_train, y_train)

            # 進行預測
            y_pred = model_instance.predict(X_test)
            
            # 取得預測機率值以計算 log_loss
            if hasattr(model_instance, 'predict_proba'):
                y_pred_proba = model_instance.predict_proba(X_test)
            else:
                print(f"    警告: 模型 {exp_model_display_name} 沒有 predict_proba 方法，log_loss 可能會受到影響。")
                # 如果無法取得機率，則使用均勻分佈機率
                y_pred_proba = np.full((len(y_test), len(np.unique(y))), 1/len(np.unique(y)))

            # 計算性能指標
            accuracy = accuracy_score(y_test, y_pred)
            f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)
            f1_weighted = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            prec_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)
            recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)
            
            # 計算對數損失 (log_loss)
            try:
                current_loss = log_loss(y_test, y_pred_proba, labels=np.unique(y))
            except ValueError as e:
                print(f"    警告：無法為模型 {exp_model_display_name} 在 Fold {fold} 計算 log_loss: {e}")
                current_loss = np.nan

            end_time_fold = time.time()
            time_sec = round(end_time_fold - start_time_fold, 2)

            # 儲存每個摺疊的結果和指標
            fold_row = {
                'fold': fold,
                'exp': f'{exp_model_display_name}_{mode_name}', # 實驗名稱現在使用映射後的顯示名稱
                'best_macro_f1': round(f1_macro, 15),
                'time_sec': time_sec,
                'acc': round(accuracy, 15),
                'f1_macro': round(f1_macro, 15),
                'f1_weighted': round(f1_weighted, 15),
                'prec_macro': round(prec_macro, 15),
                'recall_macro': round(recall_macro, 15),
                'loss': round(current_loss, 15),
                'original_model_key': model_key, # 儲存原始模型鍵用於排序
                'mode_name_key': mode_name # 儲存模式鍵用於排序
            }
            temp_results_per_fold[fold].append(fold_row) # 暫存到對應的摺疊列表中

            print(f"    摺疊 {fold} 完成。F1-Macro: {f1_macro:.4f}, 準確率: {accuracy:.4f}, 耗時: {time_sec:.2f}秒, 損失: {current_loss:.4f}")

# 現在，將所有摺疊的結果按照您指定的順序組裝到最終列表中
print("\n--------------------------------------------------")
print("開始組裝最終結果列表...")

# 自定義排序函數，用於 `exp` 欄位 (現在直接使用儲存的鍵)
def custom_exp_sort_key(row_dict):
    original_model_key = row_dict['original_model_key']
    mode_name_key = row_dict['mode_name_key']
    
    # 獲取模型和模式的排序值，使用明確的列表長度作為預設值
    model_order = model_sort_values.get(original_model_key, len(model_display_order_list))
    mode_order = mode_sort_values.get(mode_name_key, len(mode_display_order_list))
    
    return (model_order, mode_order)

# 遍歷每個摺疊，並在每個摺疊內部按照 custom_exp_sort_key 排序
for fold_num in sorted(temp_results_per_fold.keys()):
    # 對當前摺疊的結果進行排序
    sorted_fold_results = sorted(temp_results_per_fold[fold_num], key=lambda x: custom_exp_sort_key(x))
    all_results.extend(sorted_fold_results) # 將排序後的結果添加到最終列表

print("最終結果列表組裝完成。")
print("\n--- 分類結果（英文新聞）---")

results_df = pd.DataFrame(all_results)
results_df = results_df.drop(columns=['original_model_key', 'mode_name_key'])
results_df.to_csv('results_summary.csv', index=False, float_format='%.15f')
print(results_df.head())  


開始載入數據並準備英文文本（標題與標題+內容）...
數據載入與英文文本準備完成。共 3166 篇新聞。
--------------------------------------------------
開始對每個文本模式和每個模型執行 5 摺分層交叉驗證...

--- 正在處理模型: Logistic Regression, 文本模式: title ---
  開始建立 TF-IDF 向量（針對 title 文本）...
  TF-IDF 向量建立完成。詞彙量: 1979
  特徵矩陣形狀（文件數, 特徵數）: (3166, 1979)
  --------------------------------------------------
    摺疊 0 完成。F1-Macro: 0.6101, 準確率: 0.6562, 耗時: 0.02秒, 損失: 0.8208
    摺疊 1 完成。F1-Macro: 0.6186, 準確率: 0.6651, 耗時: 0.02秒, 損失: 0.7984
    摺疊 2 完成。F1-Macro: 0.5747, 準確率: 0.6367, 耗時: 0.02秒, 損失: 0.8143
    摺疊 3 完成。F1-Macro: 0.5694, 準確率: 0.6256, 耗時: 0.02秒, 損失: 0.8310
    摺疊 4 完成。F1-Macro: 0.6194, 準確率: 0.6682, 耗時: 0.02秒, 損失: 0.7877

--- 正在處理模型: Logistic Regression, 文本模式: title+content ---
  開始建立 TF-IDF 向量（針對 title+content 文本）...
  TF-IDF 向量建立完成。詞彙量: 9242
  特徵矩陣形狀（文件數, 特徵數）: (3166, 9242)
  --------------------------------------------------
    摺疊 0 完成。F1-Macro: 0.6494, 準確率: 0.6940, 耗時: 0.10秒, 損失: 0.7627
    摺疊 1 完成。F1-Macro: 0.6720, 準確率: 0.7204, 耗時: 0.12秒, 損失: 0.7293
    摺